In [2]:
import pandas as pd
import numpy as np
import os

results_dir = "[2] Analyzer Results"

recording_order = [15, 2, 1, 6, 10, 4]# interpulse intervals

IPI = {

    "ASR_control":      None,
    "gap_depth":        50,
    "gap_duration_10":  10,
    "gap_duration_20":  20,
    "gap_duration_4":   4,
    "gap_duration_50":  50,
    "gap_duration_8":   8,
    "offset_ASR":       None,
    "offset_PPI_10":    None,
    "offset_PPI_12":    None,
    "offset_PPI_14":    None,
    "offset_PPI_16":    None,
    "offset_PPI_18":    None,
    "offset_PPI_20":    None,
    "offset_PPI_4":     None,
    "offset_PPI_6":     None,
    "offset_PPI_8":     None,
    "offset_PPI_50":    None,
    "tone_in_noise":    50

}
prepulse_intensity = {

    "ASR_control":      120,
    "gap_depth":        30,
    "gap_duration_10":  60,
    "gap_duration_20":  60,
    "gap_duration_4":   60,
    "gap_duration_50":  60,
    "gap_duration_8":   60,
    "offset_ASR":       0,
    "offset_PPI_10":    0,
    "offset_PPI_12":    0,
    "offset_PPI_14":    0,
    "offset_PPI_16":    0,
    "offset_PPI_18":    0,
    "offset_PPI_20":    0,
    "offset_PPI_4":     0,
    "offset_PPI_6":     0,
    "offset_PPI_8":     0,
    "offset_PPI_50":    0,
    "tone_in_noise":    80
}
# ...existing code...

dfs = {}
dfs_fit = {}

files = [file for file in os.listdir(results_dir) if file.endswith(('.xlsx', '.xls'))]

for file in files:
    file_path = os.path.join(results_dir, file)
    key = file.split('.')[0]
    if key.endswith('_FIT'):
        dfs_fit[key] = pd.read_excel(file_path)
    else:
        dfs[key] = pd.read_excel(file_path)

print("Raw dataframes:", list(dfs.keys()))
print("Fit dataframes:", list(dfs_fit.keys()))
print(f"Found {len(dfs)} raw and {len(dfs_fit)} fit dataframes in {results_dir}.")
if len(dfs) != len(dfs_fit):
    print("Warning: Mismatch between raw and fit dataframes!")

experiments = dfs[list(dfs.keys())[1]]['experiment'].unique().tolist()
variables = dfs[list(dfs.keys())[1]].columns[4:].tolist()
parameters = dfs[list(dfs.keys())[1]].columns[:4].tolist()
print(f"\nFound {len(experiments)} experiments, {len(variables)} variables and {len(parameters)} parameters:")
print(" "+', '.join(experiments))
print(" "+', '.join(variables))
print(" "+', '.join(parameters))

Raw dataframes: ['RESULTS', 'RESULTS_MERGED', 'RESULTS_MERGED_DATE', 'RESULTS_MERGED_EXP', 'RESULTS_MT10', 'RESULTS_MTT_MERGED', 'RESULTS_MTT_MERGED_DATE', 'RESULTS_MTT_MERGED_EXP', 'RESULTS_T10', 'RESULTS_TT_MERGED', 'RESULTS_TT_MERGED_DATE', 'RESULTS_TT_MERGED_EXP', 'SegmentOrder']
Fit dataframes: []
Found 13 raw and 0 fit dataframes in [2] Analyzer Results.

Found 18 experiments, 9 variables and 4 parameters:
 ASR_control, gap_depth, tone_in_noise, gap_duration_4, gap_duration_8, gap_duration_10, gap_duration_20, gap_duration_50, offset_PPI_4, offset_PPI_6, offset_PPI_8, offset_PPI_10, offset_PPI_12, offset_PPI_14, offset_PPI_16, offset_PPI_18, offset_PPI_20, offset_PPI_50
 reactionTime, peakTime, difference, peakValue, PTPA, PTTA, RMS, tau, AUC
 animal, sex, date, experiment


---
Expectations:
- date fluctuations
- no repetition differences
- sex differences in strength metrics
- experiment differences, in particular with increased gap / offset
---

In [3]:
# get S from ASR Control
# get PPS from PPI experiments
# get PPSoffset from offset experiments

#calculate %PPI and %PPIoffset

#split all by date and sex

In [4]:
# --- ASR_control ---
asr_data_ctrl = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        asr_control = df[df['experiment'] == 'ASR_control'].copy()
        asr_control['peakValue'] = pd.to_numeric(asr_control['peakValue'], errors='coerce')
        asr_data_ctrl.append(asr_control[['date', 'sex', 'peakValue']])

ctrl_dict = {}
if asr_data_ctrl:
    asr_df = pd.concat(asr_data_ctrl, ignore_index=True)
    avg_by_date_sex = asr_df.groupby(['date', 'sex'])['peakValue'].mean().reset_index()
    print("Average peakValue from ASR_control experiments split by date and sex:")
    #print(avg_by_date_sex)
    overall_avg = asr_df['peakValue'].mean()
    print(f"Overall average peakValue from ASR_control experiments: {overall_avg:.2f}")
    ctrl_dict = {(row['date'], row['sex']): row['peakValue'] for _, row in avg_by_date_sex.iterrows()}
else:
    print("No ASR_control experiments with date, sex, and peakValue found.")


# --- tone_in_noise ---
asr_ttn = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        ttn_control = df[df['experiment'] == 'tone_in_noise'].copy()
        ttn_control['peakValue'] = pd.to_numeric(ttn_control['peakValue'], errors='coerce')
        asr_ttn.append(ttn_control[['date', 'sex', 'peakValue']])

ttn_dict = {}
if asr_ttn:
    ttn_df = pd.concat(asr_ttn, ignore_index=True)
    avg_by_date_sex = ttn_df.groupby(['date', 'sex'])['peakValue'].mean().reset_index()
    print("\nAverage peakValue from tone_in_noise experiments split by date and sex:")
    #print(avg_by_date_sex)
    overall_avg = ttn_df['peakValue'].mean()
    print(f"Overall average peakValue from tone_in_noise experiments: {overall_avg:.2f}")
    ttn_dict = {(row['date'], row['sex']): row['peakValue'] for _, row in avg_by_date_sex.iterrows()}
else:
    print("\nNo tone_in_noise experiments with date, sex, and peakValue found.")


# --- Average peakValue across selected experiments (excluding ASR_control, tone_in_noise, offset_ASR, gap_depth) ---
excluded_experiments = {'ASR_control', 'tone_in_noise', 'offset_ASR', 'gap_depth'}
multi_exp_data = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        mask = ~df['experiment'].isin(excluded_experiments)
        selected = df[mask].copy()
        selected['peakValue'] = pd.to_numeric(selected['peakValue'], errors='coerce')
        multi_exp_data.append(selected[['experiment', 'date', 'sex', 'peakValue']])

mix_dict = {}
if multi_exp_data:
    multi_exp_df = pd.concat(multi_exp_data, ignore_index=True)
    avg_by_exp_date_sex = multi_exp_df.groupby(['experiment', 'date', 'sex'])['peakValue'].mean().reset_index()
    print("\nAverage peakValue from selected experiments split by experiment, date, and sex:")
    #print(avg_by_exp_date_sex)
    overall_avg = multi_exp_df['peakValue'].mean()
    print(f"Overall average peakValue from selected experiments: {overall_avg:.2f}")
    mix_dict = {(row['experiment'], row['date'], row['sex']): row['peakValue'] for _, row in avg_by_exp_date_sex.iterrows()}
else:
    print("\nNo selected experiments with date, sex, and peakValue found.")


# --- Average peakValue for gap_duration experiments ---
gap_duration_data = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        gap_mask = df['experiment'].str.startswith('gap_duration')
        gap_df = df[gap_mask].copy()
        gap_df['peakValue'] = pd.to_numeric(gap_df['peakValue'], errors='coerce')
        gap_duration_data.append(gap_df[['experiment', 'date', 'sex', 'peakValue']])

gap_dict = {}
if gap_duration_data:
    gap_duration_df = pd.concat(gap_duration_data, ignore_index=True)
    avg_by_gap_date_sex = gap_duration_df.groupby(['experiment', 'date', 'sex'])['peakValue'].mean().reset_index()
    print("\nAverage peakValue from gap_duration experiments split by experiment, date, and sex:")
    #print(avg_by_gap_date_sex)
    overall_avg = gap_duration_df['peakValue'].mean()
    print(f"Overall average peakValue from gap_duration experiments: {overall_avg:.2f}")
    gap_dict = {(row['experiment'], row['date'], row['sex']): row['peakValue'] for _, row in avg_by_gap_date_sex.iterrows()}
else:
    print("\nNo gap_duration experiments with date, sex, and peakValue found.")


# --- Average peakValue for offset_PPI experiments ---
offset_ppi_data = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        offset_mask = df['experiment'].str.startswith('offset_PPI')
        offset_df = df[offset_mask].copy()
        offset_df['peakValue'] = pd.to_numeric(offset_df['peakValue'], errors='coerce')
        offset_ppi_data.append(offset_df[['experiment', 'date', 'sex', 'peakValue']])

offset_dict = {}
if offset_ppi_data:
    offset_ppi_df = pd.concat(offset_ppi_data, ignore_index=True)
    avg_by_offset_date_sex = offset_ppi_df.groupby(['experiment', 'date', 'sex'])['peakValue'].mean().reset_index()
    print("\nAverage peakValue from offset_PPI experiments split by experiment, date, and sex:")
    #print(avg_by_offset_date_sex)
    overall_avg = offset_ppi_df['peakValue'].mean()
    print(f"Overall average peakValue from offset_PPI experiments: {overall_avg:.2f}")
    offset_dict = {(row['experiment'], row['date'], row['sex']): row['peakValue'] for _, row in avg_by_offset_date_sex.iterrows()}
else:
    print("\nNo offset_PPI experiments with date, sex, and peakValue found.")



print()
print(ctrl_dict)
print(ttn_dict)
print(mix_dict)
print(gap_dict)
print(offset_dict)


Average peakValue from ASR_control experiments split by date and sex:
Overall average peakValue from ASR_control experiments: 132.44

Average peakValue from tone_in_noise experiments split by date and sex:
Overall average peakValue from tone_in_noise experiments: 102.65

Average peakValue from selected experiments split by experiment, date, and sex:
Overall average peakValue from selected experiments: 127.01

Average peakValue from gap_duration experiments split by experiment, date, and sex:
Overall average peakValue from gap_duration experiments: 136.42

Average peakValue from offset_PPI experiments split by experiment, date, and sex:
Overall average peakValue from offset_PPI experiments: 122.38

{('April16', 'female'): 79.14285714285714, ('April16', 'male'): 134.01666666666668, ('June26', 'female'): 121.25, ('June26', 'male'): 207.6, ('May20', 'female'): 96.86875, ('May20', 'male'): 173.18571428571428}
{('April16', 'female'): 52.55208333333333, ('April16', 'male'): 110.67619047619048

In [5]:
from scipy.stats import ttest_ind

# --- Calculate overall %PPI for each experiment type ---
def calc_overall_percent_ppi(ctrl_dict, exp_dict, label):
    ppi_values = []
    for key, PPS in exp_dict.items():
        if isinstance(key, tuple) and len(key) == 2:
            date, sex = key
        elif isinstance(key, tuple) and len(key) == 3:
            _, date, sex = key
        else:
            continue
        S = ctrl_dict.get((date, sex), np.nan)
        if pd.notna(S) and pd.notna(PPS) and S != 0:
            ppi = 100 * (S - PPS) / S
            ppi_values.append(ppi)
    if ppi_values:
        overall_ppi = np.nanmean(ppi_values)
        print(f"\nOverall %PPI for {label}: {overall_ppi:.2f}")
    else:
        print(f"\nNo valid %PPI values for {label}.")
    return ppi_values

# Usage for each experiment type
ppi_ttn_vals = calc_overall_percent_ppi(ctrl_dict, ttn_dict, "tone_in_noise")
ppi_mix_vals = calc_overall_percent_ppi(ctrl_dict, mix_dict, "selected experiments")
ppi_gap_vals = calc_overall_percent_ppi(ctrl_dict, gap_dict, "gap_duration")
ppi_offset_vals = calc_overall_percent_ppi(ctrl_dict, offset_dict, "offset_PPI")

# --- Statistical significance between groups ---
print("\nSignificance tests (t-test, unequal variance):")
groups = {
    "tone_in_noise": ppi_ttn_vals,
    "selected experiments": ppi_mix_vals,
    "gap_duration": ppi_gap_vals,
    "offset_PPI": ppi_offset_vals
}

labels = list(groups.keys())
for i in range(len(labels)):
    for j in range(i+1, len(labels)):
        group1 = groups[labels[i]]
        group2 = groups[labels[j]]
        # Only test if both groups have at least 2 values
        if len(group1) > 1 and len(group2) > 1:
            stat, pval = ttest_ind(group1, group2, nan_policy='omit', equal_var=False)
            print(f"{labels[i]} vs {labels[j]}: p = {pval:.4f}")
        else:
            print(f"{labels[i]} vs {labels[j]}: Not enough data for t-test.")


Overall %PPI for tone_in_noise: 25.48

Overall %PPI for selected experiments: 6.95

Overall %PPI for gap_duration: 0.26

Overall %PPI for offset_PPI: 10.30

Significance tests (t-test, unequal variance):
tone_in_noise vs selected experiments: p = 0.0052
tone_in_noise vs gap_duration: p = 0.0008
tone_in_noise vs offset_PPI: p = 0.0123
selected experiments vs gap_duration: p = 0.0023
selected experiments vs offset_PPI: p = 0.0352
gap_duration vs offset_PPI: p = 0.0000


In [6]:
def calc_percent_ppi(ctrl_dict, exp_dict, label):
    # Average %PPI per (experiment, date, sex) or (date, sex)
    percent_ppi = {}
    values_by_group = {}
    for key, PPS in exp_dict.items():
        if isinstance(key, tuple) and len(key) == 2:
            # (date, sex)
            group_key = key
            date, sex = key
        elif isinstance(key, tuple) and len(key) == 3:
            # (experiment, date, sex)
            group_key = key
            _, date, sex = key
        else:
            continue
        S = ctrl_dict.get((date, sex), np.nan)
        if pd.notna(S) and pd.notna(PPS) and S != 0:
            ppi_val = 100 * (S - PPS) / S
        else:
            ppi_val = np.nan
        if group_key not in values_by_group:
            values_by_group[group_key] = []
        values_by_group[group_key].append(ppi_val)
    # Average over all values for each group
    for group_key, vlist in values_by_group.items():
        vlist = [v for v in vlist if pd.notna(v)]
        percent_ppi[group_key] = np.nanmean(vlist) if vlist else np.nan
    print(f"\n%PPI for {label} (averaged per experiment, date, sex):")
    for k, v in percent_ppi.items():
        print(f"{k}: {v:.2f}" if pd.notna(v) else f"{k}: NaN")
    return percent_ppi


ppi_offset = calc_percent_ppi(ctrl_dict, offset_dict, "offset_PPI")
ppi_gap = calc_percent_ppi(ctrl_dict, gap_dict, "gap_duration")
ppi_ttn = calc_percent_ppi(ctrl_dict, ttn_dict, "tone_in_noise")


%PPI for offset_PPI (averaged per experiment, date, sex):
('offset_PPI_10', 'April16', 'female'): 21.87
('offset_PPI_10', 'April16', 'male'): 15.87
('offset_PPI_10', 'June26', 'female'): 10.73
('offset_PPI_10', 'June26', 'male'): 10.75
('offset_PPI_10', 'May20', 'female'): -5.73
('offset_PPI_10', 'May20', 'male'): 5.48
('offset_PPI_12', 'April16', 'female'): 22.45
('offset_PPI_12', 'April16', 'male'): 9.16
('offset_PPI_12', 'June26', 'female'): 12.85
('offset_PPI_12', 'June26', 'male'): 14.10
('offset_PPI_12', 'May20', 'female'): 10.15
('offset_PPI_12', 'May20', 'male'): 6.15
('offset_PPI_14', 'April16', 'female'): 22.71
('offset_PPI_14', 'April16', 'male'): -12.73
('offset_PPI_14', 'June26', 'female'): 2.76
('offset_PPI_14', 'June26', 'male'): 15.16
('offset_PPI_14', 'May20', 'female'): 14.35
('offset_PPI_14', 'May20', 'male'): 6.15
('offset_PPI_16', 'April16', 'female'): 27.66
('offset_PPI_16', 'April16', 'male'): 7.66
('offset_PPI_16', 'June26', 'female'): 16.05
('offset_PPI_16', '

In [7]:
# --- Test effect of gap_duration/offset_PPI trailing number on %PPI, split by experiment type only ---

from scipy.stats import kruskal
import re

def extract_trailing_number(experiment_name):
    match = re.search(r'(\d+)$', experiment_name)
    return int(match.group(1)) if match else None

def collect_ppi_by_param(exp_dict, ctrl_dict, param_type):
    # Collect raw %PPI values by trailing number for each experiment type (ignoring date/sex)
    grouped = {}
    for key in exp_dict:
        if isinstance(key, tuple) and len(key) == 3:
            experiment, date, sex = key
            if experiment.startswith(param_type):
                num = extract_trailing_number(experiment)
                S = ctrl_dict.get((date, sex), np.nan)
                PPS = exp_dict[key]
                if num is not None and pd.notna(S) and pd.notna(PPS) and S != 0:
                    ppi_val = 100 * (S - PPS) / S
                    grouped.setdefault(num, []).append(ppi_val)
    return grouped

# Collect raw %PPI values by trailing number (all dates/sex pooled)
gap_raw_groups = collect_ppi_by_param(gap_dict, ctrl_dict, 'gap_duration')
offset_raw_groups = collect_ppi_by_param(offset_dict, ctrl_dict, 'offset_PPI')

def test_and_report(groups, label):
    print(f"\nEffect of {label} (trailing number) on %PPI (all dates/sex pooled):")
    for k in sorted(groups):
        vals = groups[k]
        print(f"{label} {k}: n={len(vals)}, mean={np.mean(vals):.2f}, median={np.median(vals):.2f}")
    valid_groups = [v for v in groups.values() if len(v) > 1]
    if len(valid_groups) > 1:
        stat, p = kruskal(*valid_groups)
        print(f"Kruskal-Wallis H-test p-value: {p:.4f}")
        if p < 0.05:
            all_vals = [item for sublist in valid_groups for item in sublist]
            group_means = [np.mean(v) for v in valid_groups]
            grand_mean = np.mean(all_vals)
            ss_between = sum([len(v) * (m - grand_mean)**2 for v, m in zip(valid_groups, group_means)])
            ss_total = sum([(x - grand_mean)**2 for x in all_vals])
            eta_sq = ss_between / ss_total if ss_total > 0 else np.nan
            print(f"Significant effect! Effect strength (eta²): {eta_sq:.2f}")
        else:
            print("No significant effect of duration/offset on %PPI.")
    else:
        print("Not enough data for Kruskal-Wallis test.")

# Run for gap_duration and offset_PPI (pooled by experiment type)
test_and_report(gap_raw_groups, "gap_duration")
test_and_report(offset_raw_groups, "offset_PPI")


Effect of gap_duration (trailing number) on %PPI (all dates/sex pooled):
gap_duration 4: n=6, mean=-5.21, median=-6.07
gap_duration 8: n=6, mean=1.77, median=-2.86
gap_duration 10: n=6, mean=2.53, median=4.20
gap_duration 20: n=6, mean=2.26, median=3.77
gap_duration 50: n=6, mean=-0.07, median=0.54
Kruskal-Wallis H-test p-value: 0.7563
No significant effect of duration/offset on %PPI.

Effect of offset_PPI (trailing number) on %PPI (all dates/sex pooled):
offset_PPI 4: n=6, mean=7.72, median=9.98
offset_PPI 6: n=6, mean=10.19, median=7.12
offset_PPI 8: n=6, mean=14.50, median=15.17
offset_PPI 10: n=6, mean=9.83, median=10.74
offset_PPI 12: n=6, mean=12.48, median=11.50
offset_PPI 14: n=6, mean=8.07, median=10.25
offset_PPI 16: n=6, mean=10.95, median=11.33
offset_PPI 18: n=6, mean=11.63, median=9.10
offset_PPI 20: n=6, mean=10.76, median=10.50
offset_PPI 50: n=6, mean=6.90, median=6.45
Kruskal-Wallis H-test p-value: 0.9602
No significant effect of duration/offset on %PPI.


In [8]:
def average_ppi_by_sex(ppi_dict, label):
    # Collect values by sex
    
    print(f"\nAverage %PPI across dates for {label}:")
    PPF_count = 0
    PPI_count = 0
    PPF = {}
    values_by_sex = {}
    for key, val in ppi_dict.items():
        if isinstance(key, tuple):
            sex = key[-1]  # works for (date, sex) or (experiment, date, sex)
            if pd.notna(val):
                values_by_sex.setdefault(sex, []).append(val)
                if val < 0:
                    PPF_count += 1
                    PPF[key] = val
                else:
                    PPI_count += 1
    print(f"PPI count: {PPI_count}")
    # Average for each sex
    
    avg_by_sex = {sex: np.nanmean(vals) if vals else np.nan for sex, vals in values_by_sex.items()}
    for sex, avg in avg_by_sex.items():
        print(f"{sex}: {avg:.2f}" if pd.notna(avg) else f"{sex}: NaN")
    print(f"mean: {np.mean(list(avg_by_sex.values())):.2f}")
    print(f"PPF count: {PPF_count}")
    for k, v in PPF.items():
        #print(f"{k}: {v:.2f}")
        pass
    print(f"PPF mean: {np.mean(list(PPF.values())):.2f}" if PPF else "PPF mean: NaN")
    return None

# Example usage:

average_ppi_by_sex(ppi_offset, "offset_PPI")
average_ppi_by_sex(ppi_gap, "gap_duration")
average_ppi_by_sex(ppi_ttn, "tone_in_noise")


Average %PPI across dates for offset_PPI:
PPI count: 54
female: 11.60
male: 9.01
mean: 10.30
PPF count: 6
PPF mean: -7.62

Average %PPI across dates for gap_duration:
PPI count: 15
female: 0.59
male: -0.08
mean: 0.26
PPF count: 15
PPF mean: -7.47

Average %PPI across dates for tone_in_noise:
PPI count: 6
female: 30.05
male: 20.91
mean: 25.48
PPF count: 0
PPF mean: NaN


In [9]:
print("\nNumber of negative %PPI cases for gap_duration experiments:")
gap_keys = set(gap_dict.keys())
negative_count = 0
total_count = 0

for key in gap_keys:
    experiment, date, sex = key
    PPS = gap_dict.get(key)
    S = ctrl_dict.get((date, sex), np.nan)
    if pd.notna(S) and pd.notna(PPS) and S != 0:
        ppi = 100 * (S - PPS) / S
        total_count += 1
        if ppi < 0:
            negative_count += 1

print(f"Negative %PPI cases: {negative_count} out of {total_count}")


Number of negative %PPI cases for gap_duration experiments:
Negative %PPI cases: 15 out of 30


In [10]:
from collections import Counter

print("\nChecking for trends in negative %PPI cases for gap_duration experiments:")

gap_keys = set(gap_dict.keys())
negative_dates = []
negative_sexes = []

for key in gap_keys:
    experiment, date, sex = key
    PPS = gap_dict.get(key)
    S = ctrl_dict.get((date, sex), np.nan)
    if pd.notna(S) and pd.notna(PPS) and S != 0:
        ppi = 100 * (S - PPS) / S
        if ppi < 0:
            negative_dates.append(date)
            negative_sexes.append(sex)

date_counts = Counter(negative_dates)
sex_counts = Counter(negative_sexes)

print("Negative %PPI by date:", dict(date_counts))
print("Negative %PPI by sex:", dict(sex_counts))


Checking for trends in negative %PPI cases for gap_duration experiments:
Negative %PPI by date: {'June26': 4, 'April16': 3, 'May20': 8}
Negative %PPI by sex: {'male': 8, 'female': 7}


In [11]:
print("\n%PPI for each gap_duration experiment:")
gap_keys = set(gap_dict.keys())

for key in sorted(gap_keys):
    # key is (experiment, date, sex)
    experiment, date, sex = key
    PPS = gap_dict.get(key)
    S = ctrl_dict.get((date, sex), np.nan)
    if pd.notna(S) and pd.notna(PPS) and S != 0:
        ppi = 100 * (S - PPS) / S
        print(f"{key}: %PPI = {ppi:.2f}")
    else:
        print(f"{key}: %PPI = NaN")


%PPI for each gap_duration experiment:
('gap_duration_10', 'April16', 'female'): %PPI = 10.76
('gap_duration_10', 'April16', 'male'): %PPI = 4.99
('gap_duration_10', 'June26', 'female'): %PPI = -0.67
('gap_duration_10', 'June26', 'male'): %PPI = 3.42
('gap_duration_10', 'May20', 'female'): %PPI = -8.45
('gap_duration_10', 'May20', 'male'): %PPI = 5.12
('gap_duration_20', 'April16', 'female'): %PPI = 4.55
('gap_duration_20', 'April16', 'male'): %PPI = 9.46
('gap_duration_20', 'June26', 'female'): %PPI = 3.00
('gap_duration_20', 'June26', 'male'): %PPI = -1.55
('gap_duration_20', 'May20', 'female'): %PPI = -12.50
('gap_duration_20', 'May20', 'male'): %PPI = 10.58
('gap_duration_4', 'April16', 'female'): %PPI = 12.13
('gap_duration_4', 'April16', 'male'): %PPI = -10.30
('gap_duration_4', 'June26', 'female'): %PPI = -6.32
('gap_duration_4', 'June26', 'male'): %PPI = 2.33
('gap_duration_4', 'May20', 'female'): %PPI = -23.29
('gap_duration_4', 'May20', 'male'): %PPI = -5.81
('gap_duration_5

In [12]:
def calc_percent_ppi(ctrl_dict, exp_dict, label):
    # If exp_dict keys include experiment, average over experiments for each (date, sex)
    percent_ppi = {}
    # Collect values by (date, sex)
    values_by_date_sex = {}
    for key, PPS in exp_dict.items():
        if isinstance(key, tuple) and len(key) == 2:
            date, sex = key
        elif isinstance(key, tuple) and len(key) == 3:
            _, date, sex = key
        else:
            continue
        S = ctrl_dict.get((date, sex), np.nan)
        if pd.notna(S) and pd.notna(PPS) and S != 0:
            ppi_val = 100 * (S - PPS) / S
        else:
            ppi_val = np.nan
        # Group by (date, sex)
        if (date, sex) not in values_by_date_sex:
            values_by_date_sex[(date, sex)] = []
        values_by_date_sex[(date, sex)].append(ppi_val)
    # Average over experiments for each (date, sex)
    for k, vlist in values_by_date_sex.items():
        vlist = [v for v in vlist if pd.notna(v)]
        percent_ppi[k] = np.nanmean(vlist) if vlist else np.nan
    print(f"\n%PPI for {label} (averaged over experiments if multiple):")
    for k, v in percent_ppi.items():
        print(f"{k}: {v:.2f}" if pd.notna(v) else f"{k}: NaN")
    return percent_ppi

# Calculate %PPI for each experiment type
ppi_ttn = calc_percent_ppi(ctrl_dict, ttn_dict, "tone_in_noise")
ppi_mix = calc_percent_ppi(ctrl_dict, mix_dict, "selected experiments")
ppi_gap = calc_percent_ppi(ctrl_dict, gap_dict, "gap_duration")
ppi_offset = calc_percent_ppi(ctrl_dict, offset_dict, "offset_PPI")


%PPI for tone_in_noise (averaged over experiments if multiple):
('April16', 'female'): 33.60
('April16', 'male'): 17.42
('June26', 'female'): 41.15
('June26', 'male'): 24.51
('May20', 'female'): 15.41
('May20', 'male'): 20.79

%PPI for selected experiments (averaged over experiments if multiple):
('April16', 'female'): 18.89
('April16', 'male'): 6.62
('June26', 'female'): 7.92
('June26', 'male'): 5.45
('May20', 'female'): -3.03
('May20', 'male'): 5.87

%PPI for gap_duration (averaged over experiments if multiple):
('April16', 'female'): 12.86
('April16', 'male'): -1.40
('June26', 'female'): 1.33
('June26', 'male'): 1.62
('May20', 'female'): -12.43
('May20', 'male'): -0.45

%PPI for offset_PPI (averaged over experiments if multiple):
('April16', 'female'): 21.90
('April16', 'male'): 10.63
('June26', 'female'): 11.21
('June26', 'male'): 7.36
('May20', 'female'): 1.67
('May20', 'male'): 9.03


In [13]:
def count_negative_ppi(ppi_dict, label):
    negatives = [v for v in ppi_dict.values() if pd.notna(v) and v < 0]
    print(f"\nNumber of negative %PPI cases for {label}: {len(negatives)}")
    return len(negatives)

# Count negative %PPI cases for each
count_negative_ppi(ppi_ttn, "tone_in_noise")
#count_negative_ppi(ppi_mix, "selected experiments")
count_negative_ppi(ppi_gap, "gap_duration")
count_negative_ppi(ppi_offset, "offset_PPI")


Number of negative %PPI cases for tone_in_noise: 0

Number of negative %PPI cases for gap_duration: 3

Number of negative %PPI cases for offset_PPI: 0


0

In [14]:
print("\nDifferences between %PPI groups per (date, sex):")
group_dicts = [ppi_ttn, ppi_gap, ppi_offset]
group_labels = ["tone_in_noise", "gap_duration", "offset_PPI"]

# Get all unique (date, sex) pairs present in any group
all_keys = set()
for d in group_dicts:
    all_keys.update(d.keys())

for key in sorted(all_keys):
    vals = []
    for d in group_dicts:
        v = d.get(key)
        vals.append(v if pd.notna(v) else None)
    # Only compare if at least two groups have values for this (date, sex)
    for i in range(len(vals)):
        for j in range(i+1, len(vals)):
            v1 = vals[i]
            v2 = vals[j]
            if v1 is not None and v2 is not None:
                # t-test not meaningful for single values, so just compare directly
                # If you have replicates per group, you can use arrays and t-test
                # Here, just check difference and print p-value as NA
                if v1 != v2:
                    mean_diff = v1 - v2
                    print(f"{key}: {group_labels[i]} ({v1:.2f}) vs {group_labels[j]} ({v2:.2f}) | Mean diff: {mean_diff:.2f}")


Differences between %PPI groups per (date, sex):
('April16', 'female'): tone_in_noise (33.60) vs gap_duration (12.86) | Mean diff: 20.74
('April16', 'female'): tone_in_noise (33.60) vs offset_PPI (21.90) | Mean diff: 11.69
('April16', 'female'): gap_duration (12.86) vs offset_PPI (21.90) | Mean diff: -9.04
('April16', 'male'): tone_in_noise (17.42) vs gap_duration (-1.40) | Mean diff: 18.81
('April16', 'male'): tone_in_noise (17.42) vs offset_PPI (10.63) | Mean diff: 6.78
('April16', 'male'): gap_duration (-1.40) vs offset_PPI (10.63) | Mean diff: -12.03
('June26', 'female'): tone_in_noise (41.15) vs gap_duration (1.33) | Mean diff: 39.82
('June26', 'female'): tone_in_noise (41.15) vs offset_PPI (11.21) | Mean diff: 29.93
('June26', 'female'): gap_duration (1.33) vs offset_PPI (11.21) | Mean diff: -9.88
('June26', 'male'): tone_in_noise (24.51) vs gap_duration (1.62) | Mean diff: 22.89
('June26', 'male'): tone_in_noise (24.51) vs offset_PPI (7.36) | Mean diff: 17.15
('June26', 'male')